# Statistiques sur les trains TGV de la SNCF

In [1]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('data/regularite-mensuelle-tgv-aqst.csv', delimiter=";")
df.head()

,Date,Service,Gare de départ,Gare d'arrivée,Durée moyenne du trajet,Nombre de circulations prévues,Nombre de trains annulés,Commentaire annulations,Nombre de trains en retard au départ,Retard moyen des trains en retard au départ,...,Nombre trains en retard > 15min,Retard moyen trains en retard > 15 (si liaison concurrencée par vol),Nombre trains en retard > 30min,Nombre trains en retard > 60min,Prct retard pour causes externes,Prct retard pour cause infrastructure,Prct retard pour cause gestion trafic,Prct retard pour cause matériel roulant,Prct retard pour cause gestion en gare et réutilisation de matériel,"Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)"
0,2018-01,National,ANGERS SAINT LAUD,PARIS MONTPARNASSE,91,393,2,NaN,204,6.400490,...,35,6.072847,19,4,37.500000,37.500000,8.928571,5.357143,8.928571,1.785714
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,2018-01,National,PARIS MONTPARNASSE,BORDEAUX ST JEAN,143,775,7,NaN,148,9.377815,...,45,2.546962,21,4,21.666667,25.000000,11.666667,40.000000,1.666667,0.000000
3,2018-01,National,PARIS MONTPARNASSE,LAVAL,84,217,0,NaN,32,11.609375,...,16,5.822811,5,2,24.242424,54.545455,3.030303,12.121212,3.030303,3.030303
4,2018-01,National,PARIS MONTPARNASSE,NANTES,124,508,3,NaN,71,7.235211,...,39,5.292211,18,8,33.333333,22.222222,16.666667,20.370370,5.555556,1.851852


In [81]:
df.isna().sum()

Date                                                                                              0
Service                                                                                           0
Gare de départ                                                                                    0
Gare d'arrivée                                                                                    0
Durée moyenne du trajet                                                                           0
Nombre de circulations prévues                                                                    0
Nombre de trains annulés                                                                          0
Commentaire annulations                                                                        6324
Nombre de trains en retard au départ                                                              0
Retard moyen des trains en retard au départ                                                       0


Suppression des colonnes possédant des données manquantes

In [82]:
df.dropna(axis = 1, inplace = True)
df.head()

,Date,Service,Gare de départ,Gare d'arrivée,Durée moyenne du trajet,Nombre de circulations prévues,Nombre de trains annulés,Nombre de trains en retard au départ,Retard moyen des trains en retard au départ,Retard moyen de tous les trains au départ,...,Nombre trains en retard > 15min,Retard moyen trains en retard > 15 (si liaison concurrencée par vol),Nombre trains en retard > 30min,Nombre trains en retard > 60min,Prct retard pour causes externes,Prct retard pour cause infrastructure,Prct retard pour cause gestion trafic,Prct retard pour cause matériel roulant,Prct retard pour cause gestion en gare et réutilisation de matériel,"Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)"
0,2018-01,National,ANGERS SAINT LAUD,PARIS MONTPARNASSE,91,393,2,204,6.400490,3.455754,...,35,6.072847,19,4,37.500000,37.500000,8.928571,5.357143,8.928571,1.785714
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,8,2.875000,0.095796,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,2018-01,National,PARIS MONTPARNASSE,BORDEAUX ST JEAN,143,775,7,148,9.377815,1.582530,...,45,2.546962,21,4,21.666667,25.000000,11.666667,40.000000,1.666667,0.000000
3,2018-01,National,PARIS MONTPARNASSE,LAVAL,84,217,0,32,11.609375,1.455760,...,16,5.822811,5,2,24.242424,54.545455,3.030303,12.121212,3.030303,3.030303
4,2018-01,National,PARIS MONTPARNASSE,NANTES,124,508,3,71,7.235211,0.734290,...,39,5.292211,18,8,33.333333,22.222222,16.666667,20.370370,5.555556,1.851852


In [83]:
df.describe()

,Durée moyenne du trajet,Nombre de circulations prévues,Nombre de trains annulés,Nombre de trains en retard au départ,Retard moyen des trains en retard au départ,Retard moyen de tous les trains au départ,Nombre de trains en retard à l'arrivée,Retard moyen des trains en retard à l'arrivée,Retard moyen de tous les trains à l'arrivée,Nombre trains en retard > 15min,Retard moyen trains en retard > 15 (si liaison concurrencée par vol),Nombre trains en retard > 30min,Nombre trains en retard > 60min,Prct retard pour causes externes,Prct retard pour cause infrastructure,Prct retard pour cause gestion trafic,Prct retard pour cause matériel roulant,Prct retard pour cause gestion en gare et réutilisation de matériel,"Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)"
count,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000,6324.000000
mean,168.364485,244.284472,11.543801,89.404491,10.386532,2.975748,31.871917,33.008380,5.288479,22.453985,28.313645,10.459519,3.711259,24.385401,22.869181,18.234726,18.245399,6.257706,6.481338
std,86.273859,165.248615,27.182925,91.760544,13.383193,5.835297,27.726689,15.554139,8.712921,19.353596,19.850033,9.876564,4.246828,17.445229,16.477759,14.958479,14.275646,8.157455,10.547854
min,0.000000,0.000000,0.000000,0.000000,0.000000,-229.269444,0.000000,-40.109259,-472.638889,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,99.000000,133.750000,0.000000,21.000000,4.418838,1.028414,12.000000,23.611046,2.926658,9.000000,7.739368,4.000000,1.000000,12.500000,12.121212,7.692308,8.571429,0.000000,0.000000
50%,162.000000,210.000000,3.000000,57.000000,7.965843,2.148165,25.000000,31.438492,4.681114,18.000000,31.223264,8.000000,2.000000,22.222222,20.930233,16.642764,16.666667,4.545455,2.518366
75%,219.000000,315.000000,10.000000,131.000000,12.877041,3.752811,44.000000,40.429394,7.219683,31.000000,40.984239,15.000000,5.000000,33.333333,31.034483,26.315789,25.000000,9.090909,8.333333
max,786.000000,973.000000,288.000000,591.000000,316.188095,84.516667,239.000000,255.866667,92.000000,192.000000,255.866667,91.000000,39.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


## Durée moyenne du trajet 

Afin de pouvoir visualiser nos données sur une carte, on télécharge un deuxième dataset contenant les coordonnées des gares

In [104]:
coord = pd.read_csv('data/referentiel-gares-voyageurs.csv', delimiter=";").drop(['Date fin validité plateforme', 'SOPs'], axis=1).dropna()
coord.head()

,Code plate-forme,Code gare,Code UIC,Intitulé plateforme,Code postal,Code Commune,Commune,Code département,Département,Longitude,...,Intitulé fronton de gare,Gare DRG,Gare étrangère,DTG,Région SNCF,Unité gare,UT,Nbre plateformes,TVS,WGS 84
1,00004-1,4,87785006,Cerbère,66290.0,48.0,Cerbère,66.0,Pyrénées-Orientales,3.163403,...,Cerbère,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,CERBERE GARE,1,CER,"42.4417732,3.1634033"
2,00006-1,6,87784884,Ur - Les Escaldes,66760.0,218.0,Ur,66.0,Pyrénées-Orientales,1.940482,...,Ur les Escaldes,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,UR LES ESCALDES GARE,1,URL,"42.457481,1.9404821"
3,00022-1,22,87784728,Olette - Canaveilles,66360.0,125.0,Olette,66.0,Pyrénées-Orientales,2.271931,...,Olette - Canaveilles les Bains,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,OLETTE CANAVEILLES LES BAINS GARE,1,OLE,"42.55472,2.2719309"
4,00027-1,27,87784637,Prades - Molitg-les-Bains,66500.0,149.0,Prades,66.0,Pyrénées-Orientales,2.429418,...,Prades - Molitg-les-Bains,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,PRADES MOLITG LES BAINS GARE,1,PDS,"42.6170988,2.4294184"
5,00047-1,47,87783563,Barjac,48000.0,18.0,Barjac,48.0,Lozère,3.410979,...,Barjac,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,BARJAC GARE,1,BJC,"44.4992638,3.4109786"


In [105]:
coord['UT'] = coord['UT'].astype(str)
coord['UT'] = coord['UT'].map(lambda s: s.replace(" GARE", ""))


In [106]:
coord

,Code plate-forme,Code gare,Code UIC,Intitulé plateforme,Code postal,Code Commune,Commune,Code département,Département,Longitude,...,Intitulé fronton de gare,Gare DRG,Gare étrangère,DTG,Région SNCF,Unité gare,UT,Nbre plateformes,TVS,WGS 84
1,00004-1,4,87785006,Cerbère,66290.0,48.0,Cerbère,66.0,Pyrénées-Orientales,3.163403,...,Cerbère,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,CERBERE,1,CER,"42.4417732,3.1634033"
2,00006-1,6,87784884,Ur - Les Escaldes,66760.0,218.0,Ur,66.0,Pyrénées-Orientales,1.940482,...,Ur les Escaldes,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,UR LES ESCALDES,1,URL,"42.457481,1.9404821"
3,00022-1,22,87784728,Olette - Canaveilles,66360.0,125.0,Olette,66.0,Pyrénées-Orientales,2.271931,...,Olette - Canaveilles les Bains,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,OLETTE CANAVEILLES LES BAINS,1,OLE,"42.55472,2.2719309"
4,00027-1,27,87784637,Prades - Molitg-les-Bains,66500.0,149.0,Prades,66.0,Pyrénées-Orientales,2.429418,...,Prades - Molitg-les-Bains,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,PRADES MOLITG LES BAINS,1,PDS,"42.6170988,2.4294184"
5,00047-1,47,87783563,Barjac,48000.0,18.0,Barjac,48.0,Lozère,3.410979,...,Barjac,True,False,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,BARJAC,1,BJC,"44.4992638,3.4109786"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,02991-1,2991,87141127,Rosières-aux-Salines,54110.0,462.0,Rosières-aux-Salines,54.0,Meurthe-et-Moselle,6.350457,...,Rosières-aux-Salines,True,False,DRG Grand Est,REGION LORRAINE,UG Lorraine,ROSIERES AUX SALINES P,1,ROI,"48.60563,6.350457"
3176,02993-1,2993,87141101,Varangéville - Saint-Nicolas,54110.0,549.0,Varangéville,54.0,Meurthe-et-Moselle,6.312860,...,Varangéville - Saint-Nicolas,True,False,DRG Grand Est,REGION LORRAINE,UG Lorraine,VARANGEVILLE ST NICOLAS,1,VRA,"48.63533929,6.31286037"
3177,03002-1,3002,87141002,Nancy,54000.0,395.0,Nancy,54.0,Meurthe-et-Moselle,6.174583,...,Nancy,True,False,DRG Grand Est,REGION LORRAINE,UG Lorraine,NANCY VILLE,1,NCY,"48.689861,6.174583"
3178,03020-1,3020,87116582,Château-Thierry,2400.0,168.0,Château-Thierry,2.0,Aisne,3.409514,...,Château-Thierry,True,False,DRG HDF-Normandie,REGION DE PARIS-EST,UG Picardie,CHATEAU THIERRY P,1,CTH,"49.038224,3.409514"


In [107]:
df2 = pd.merge(df, coord[['UT', 'Région SNCF', 'WGS 84']], left_on='Gare d\'arrivée', right_on='UT', how='left')

In [111]:
df2.dropna()

,Date,Service,Gare de départ,Gare d'arrivée,Durée moyenne du trajet,Nombre de circulations prévues,Nombre de trains annulés,Nombre de trains en retard au départ,Retard moyen des trains en retard au départ,Retard moyen de tous les trains au départ,...,Nombre trains en retard > 60min,Prct retard pour causes externes,Prct retard pour cause infrastructure,Prct retard pour cause gestion trafic,Prct retard pour cause matériel roulant,Prct retard pour cause gestion en gare et réutilisation de matériel,"Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)",UT,Région SNCF,WGS 84
2,2018-01,National,PARIS MONTPARNASSE,BORDEAUX ST JEAN,143,775,7,148,9.377815,1.582530,...,4,21.666667,25.000000,11.666667,40.000000,1.666667,0.000000,BORDEAUX ST JEAN,REGION POITOU-CHARENTES AQUITAINE,"44.825873,-0.556697"
3,2018-01,National,PARIS MONTPARNASSE,LAVAL,84,217,0,32,11.609375,1.455760,...,2,24.242424,54.545455,3.030303,12.121212,3.030303,3.030303,LAVAL,REGION PAYS DE LA LOIRE,"48.076206,-0.760907"
6,2018-01,National,PARIS MONTPARNASSE,TOULOUSE MATABIAU,257,182,1,38,10.412281,1.958748,...,7,47.619048,28.571429,0.000000,19.047619,4.761905,0.000000,TOULOUSE MATABIAU,REGION MIDI PYRENEES,"43.611206,1.453616"
7,2018-01,National,PARIS MONTPARNASSE,VANNES,152,253,1,38,9.746930,1.055026,...,8,25.641026,38.461538,12.820513,17.948718,0.000000,5.128205,VANNES,REGION BRETAGNE,"47.665169,-2.75245"
10,2018-01,National,LILLE,MARSEILLE ST CHARLES,307,190,4,50,7.381333,2.043459,...,7,57.142857,19.047619,4.761905,9.523810,4.761905,4.761905,MARSEILLE ST CHARLES,REGION PROVENCE ALPES COTE D'AZUR,"43.302666,5.380407"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,2022-03,National,PARIS MONTPARNASSE,LA ROCHELLE VILLE,177,217,6,26,10.530128,0.927172,...,3,31.578947,26.315789,5.263158,31.578947,0.000000,5.263158,LA ROCHELLE VILLE,REGION POITOU-CHARENTES AQUITAINE,"46.15269,-1.145305"
9275,2022-03,National,PARIS NORD,DOUAI,70,140,0,10,23.931667,1.438690,...,2,25.000000,16.666667,16.666667,8.333333,8.333333,25.000000,DOUAI,REGION NORD PAS DE CALAIS,"50.371677,3.089986"
9278,2022-03,National,SAINT ETIENNE CHATEAUCREUX,PARIS LYON,168,115,0,23,2.039130,0.845072,...,1,12.500000,12.500000,12.500000,50.000000,0.000000,12.500000,PARIS LYON,REGION DE PARIS-SUD-EST,"48.844888,2.37352"
9279,2022-03,National,SAINT ETIENNE CHATEAUCREUX,PARIS LYON,168,115,0,23,2.039130,0.845072,...,1,12.500000,12.500000,12.500000,50.000000,0.000000,12.500000,PARIS LYON,REGION DE PARIS-SUD-EST,"48.844888,2.37352"
